In [1]:
import numpy as np
import pickle
import statistics

from xgboost import XGBClassifier
from sympy.core.numbers import igcd
from datetime import datetime

from sklearn import preprocessing
from sklearn import svm
from sklearn import preprocessing
from sklearn.ensemble import IsolationForest


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler,QuantileTransformer,RobustScaler,StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score

import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import random
from scipy import stats

from pyod.models.hbos import HBOS
from pyod.models.cblof import CBLOF
from pyod.models.loci import LOCI
from pyod.models.xgbod import XGBOD
from pyod.models.cof import COF
from pyod.models.loda import LODA
from pyod.models.copod import COPOD
from pyod.models.sod import SOD
from pyod.models.vae import VAE
from pyod.models.lof import LocalOutlierFactor,LOF
from pyod.models.lscp import LSCP
from pyod.models.so_gaal import SO_GAAL
from pyod.models.mo_gaal import MO_GAAL
from pyod.models.iforest import IForest
from pyod.models.ocsvm import OCSVM
from pyod.models.auto_encoder import AutoEncoder

from keras.models import Sequential
from keras import layers
from keras.models import Model, load_model
from keras.layers import Input, Dense, Dropout,LSTM, Conv1D, MaxPooling1D, AveragePooling1D, Flatten,Reshape,UpSampling1D
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from sklearn.manifold import TSNE
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


In [2]:
# Data load
df_0=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_normal_0.csv")
df_1=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_normal_1.csv")
df_2=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_normal_2.csv")
df_3=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_normal_3.csv")
df_4=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_normal_4.csv")
df_5=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_normal_5.csv")
df_6=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_normal_6.csv")
df_mix=pd.read_csv("./Datasets/Rp3_3rd_capture/rp3_normal_all.csv")

In [3]:
# Filter vectors according to the timestamps

df_0 = df_0.loc[(df_0['timestamp'] >= 1615419435000) & (df_0['timestamp'] <= 1615451520000)]
df_1 = df_1.loc[(df_1['timestamp'] >= 1615925651000) & (df_1['timestamp'] <= 1615929569000)]
df_2 = df_2.loc[(df_2['timestamp'] >= 1615930017000) & (df_2['timestamp'] <= 1615933673000)]
df_3 = df_3.loc[(df_3['timestamp'] >= 1615969307000) & (df_3['timestamp'] <= 1615972800000)]
df_4 = df_4.loc[(df_4['timestamp'] >= 1615974874000) & (df_4['timestamp'] <= 1615978500000)]
df_5 = df_5.loc[(df_5['timestamp'] >= 1615978899000) & (df_5['timestamp'] <= 1615982700000)]
df_6 = df_6.loc[(df_6['timestamp'] >= 1615989015000) & (df_6['timestamp'] <= 1615992622000)]
df_mix = df_mix.loc[(df_mix['timestamp'] >= 1615996500000) & (df_mix['timestamp'] <= 1616005200000)]

In [4]:
df = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6, df_mix])
#df = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6])
#df= pd.concat([df_0, df_1, df_2, df_3, df_4])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5353 entries, 7 to 759
Data columns (total 82 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   time                                     5353 non-null   float64
 1   timestamp                                5353 non-null   int64  
 2   seconds                                  5353 non-null   float64
 3   connectivity                             5353 non-null   int64  
 4   alarmtimer:alarmtimer_fired              5353 non-null   int64  
 5   alarmtimer:alarmtimer_start              5353 non-null   int64  
 6   block:block_bio_backmerge                5353 non-null   int64  
 7   block:block_bio_remap                    5353 non-null   int64  
 8   block:block_dirty_buffer                 5353 non-null   int64  
 9   block:block_getrq                        5353 non-null   int64  
 10  block:block_touch_buffer                 5353 non

In [6]:
df = df.loc[(df['connectivity']==1)]

In [7]:
# Remove columns with constant values
df=df.loc[:, (df != df.iloc[0]).any()]

In [8]:
df.describe()

,time,timestamp,seconds,block:block_bio_backmerge,block:block_bio_remap,block:block_dirty_buffer,block:block_getrq,block:block_touch_buffer,block:block_unplug,clk:clk_set_rate,...,writeback:sb_clear_inode_writeback,writeback:wbc_writepage,writeback:writeback_dirty_inode,writeback:writeback_dirty_inode_enqueue,writeback:writeback_dirty_page,writeback:writeback_mark_inode_dirty,writeback:writeback_pages_written,writeback:writeback_single_inode,writeback:writeback_write_inode,writeback:writeback_written
count,5339.000000,5.339000e+03,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,...,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000,5339.000000
mean,3928.291767,1.615705e+12,5.011843,1.719798,2.635138,4.167634,0.907473,4.089530,0.504402,1.585690,...,1.074171,1.165574,54.862896,0.199850,3.126054,57.984079,0.995692,0.450272,0.233564,1.406443
std,3942.898974,2.698815e+08,0.009430,2.635422,3.840664,148.127538,1.395699,156.106611,0.772641,3.435362,...,1.885175,2.269565,181.420970,0.454724,143.379991,313.307298,0.089649,0.754349,0.649198,0.668851
min,15.033600,1.615419e+12,5.008997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,877.011500,1.615435e+12,5.010348,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,8.000000,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,1.000000
50%,1914.410000,1.615926e+12,5.010565,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,21.000000,1.000000,0.000000,0.000000,1.000000
75%,6707.675000,1.615980e+12,5.011005,5.000000,7.000000,5.000000,2.000000,0.000000,1.000000,1.000000,...,1.000000,0.000000,26.000000,0.000000,0.000000,27.000000,1.000000,1.000000,0.000000,2.000000
max,13398.100000,1.616005e+12,5.285210,59.000000,72.000000,10521.000000,27.000000,11020.000000,24.000000,14.000000,...,7.000000,27.000000,11248.000000,6.000000,10448.000000,21697.000000,3.000000,8.000000,7.000000,4.000000


In [9]:
df.drop(['time','timestamp'],axis=1,inplace=True)

In [10]:
# Split dataset in training and testing
X_train, X_test = train_test_split(df, test_size=0.10, random_state=42)
X_train.shape

(4805, 72)

In [11]:
#Save selected features
feat_list=X_train.columns

In [12]:
#Feature scaling

# Min_Max_Scaler normalization
#scaler = MinMaxScaler().fit(X_train)
#scaler = QuantileTransformer(n_quantiles=1000,random_state=42).fit(X_train)
#scaler = RobustScaler().fit(X_train)
scaler= StandardScaler().fit(X_train)
# Transform training set
X_train = scaler.transform(X_train)
# Transform test set
X_test = scaler.transform(X_test)

In [13]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

pickle.dump(X_train, open("train_data.pkl", 'wb'))
pickle.dump(X_test, open("test_data.pkl", 'wb'))

In [14]:
# Different models

#clf = IsolationForest(random_state=42, contamination=0.2)

"""detector_list=[AutoEncoder([32,16,32], hidden_activation='relu',epochs=20,contamination=0.1),
                LOF(contamination=0.1,n_neighbors=25), COPOD(contamination=0.1),
                IForest(random_state=42,contamination=0.1), CBLOF(n_clusters=20,contamination=0.1),
                HBOS(n_bins=100,contamination=0.1), LODA(n_bins=100,n_random_cuts=100,contamination=0.1)]"""
#clf = LSCP(detector_list)
#clf = AutoEncoder([16], hidden_activation='relu',epochs=20,contamination=0.2,dropout_rate=0.0,preprocessing=True)
#clf = LOF(contamination=0.05,n_neighbors=50)
#clf = CBLOF(n_clusters=20,contamination=0.2)
#clf = VAE(encoder_neurons=[32,16],decoder_neurons=[16,32],latent_dim=2,epochs=50,contamination=0.05,verbose=1)
#clf = LODA(n_bins=100,n_random_cuts=100,contamination=0.2)
#clf = COPOD(contamination=0.2)
#clf = HBOS(n_bins=100,contamination=0.2)
#clf = SO_GAAL(contamination=0.2)
#clf = MO_GAAL(contamination=0.2)
#clf = OCSVM(kernel='rbf',gamma=0.0001, nu=0.3)


"detector_list=[AutoEncoder([32,16,32], hidden_activation='relu',epochs=20,contamination=0.1),\n                LOF(contamination=0.1,n_neighbors=25), COPOD(contamination=0.1),\n                IForest(random_state=42,contamination=0.1), CBLOF(n_clusters=20,contamination=0.1),\n                HBOS(n_bins=100,contamination=0.1), LODA(n_bins=100,n_random_cuts=100,contamination=0.1)]"

In [15]:
# Model training

#clf.fit(X_train)

In [16]:
# Model evaluation with good behaviour
#pred=clf.predict(X_test)
#unique_elements, counts_elements = np.unique(pred, return_counts=True)
#print("\t",unique_elements,"    ",counts_elements)

In [17]:
#MANUAL AUTOENCODER BUILD AND TRAIN
from sklearn.cluster import DBSCAN
autoencoder = DBSCAN(eps = 3, min_samples = 5).fit(X_train)
print(autoencoder.labels_)

[ 0  0  1 ... -1  2 -1]


In [18]:
outliers = X_train[autoencoder.labels_ == -1]
print(outliers)

            0         1         2          3          4          5   \
72   -0.128329  1.236509  1.133167   0.003577   0.782247  -0.027067   
98    0.233622 -0.648581 -0.683865  -0.028450  -0.649092   0.094491   
101  -0.147878 -0.648581 -0.424289  -0.028450   0.066577  -0.027067   
102   0.186531 -0.648581 -0.683865  -0.028450  -0.649092  -0.027067   
118  -0.187381  3.498618  3.988503  67.360902  18.673983  66.951418   
...        ...       ...       ...        ...        ...        ...   
4791 -0.099557 -0.648581  0.094863  -0.028450   1.497916  -0.027067   
4792 -0.161043 -0.648581 -0.683865  -0.028450  -0.649092  -0.027067   
4801 -0.126243 -0.648581 -0.683865  -0.028450  -0.649092  -0.027067   
4802 -0.086280  1.613528  2.690623   0.003577   4.360594  -0.027067   
4804 -0.096663  1.236509  1.133167   0.003577   0.782247  -0.002755   

             6         7         8         9   ...        62        63  \
72     0.631841 -0.461051  0.127169 -1.054192  ... -0.569103 -0.511429   

In [19]:
pickle.dump(autoencoder, open("anomaly_dbscan.sav", 'wb'))